# **Utilisation de *StackAPI* pour récupérer des questions sur *Stack Overflow***

In [1]:
from stackapi import StackAPI  # API pour récupérer les questions
import pandas as pd  # Pour utiliser un dataframe
from datetime import datetime  # Utilisé lors du filtrage par date
import time  # Pour mesurer la durée d'importation

In [2]:
SITENAME = 'stackoverflow'  # Site sur lequel seront faites les requêtes
SITE = StackAPI(SITENAME)  # Instanciation de l'API

Nous allons utiliser *'search/advanced'* qui permet d'avoir plus de paramètres pour filtrer que *'questions'*. Nous nous basons sur les questions récentes (depuis le 1er janvier 2023).

[Documentation de la méthode *'search/advanced'*](https://api.stackexchange.com/docs/advanced-search)

In [3]:
SITE.page_size = 50 # Nombre souhaité de résultats. Au maximum 100, si besoin de plus, il faut incrémenter 'max_pages'
SITE.max_pages = 1 # Nombre de pages

debut = time.time()
questions = SITE.fetch('search/advanced',  # Permet d'avoir plus de paramètres pour filtrer que 'questions'
                      filter='withbody',  # Pour que le corps du message soit également récupéré
                      sort='votes',  # Classer les questions retournées par nombre de votes
                      order='desc',  # Du plus grand au plus petit
                      min=10,  # Valeur minimum requise de l'élément sur lequel le classement est basé (ici les votes)
                      views=100,  # Nombre de vues minimum
                      answers=1,  # Nombre de réponses minimum
                      fromdate=datetime(2022,1,1),  # Date de création minimum
                      todate=datetime.now()  # Date de création maximum (ici la date actuelle)
)
print(f"Durée d'importation : {(time.time() - debut):.2f} secondes.")

elements_a_garder = ['tags', 'title', 'body']  # Éléments qui nous intéressent

data = pd.DataFrame(questions['items'])[elements_a_garder]  # Création d'un dataframe avec les données récupérées
display(data)

print('Quota journalier de requêtes restant :', questions['quota_remaining'])  # Affichage du nombre restant de requêtes possibles aujourd'hui (basé sur l'adresse IP)

Durée d'importation : 0.71 secondes.


,tags,title,body
0,"[git, github, ssh, ssh-keys]",Why are connections to GitHub over SSH throwin...,<p>Just sometime ago I started getting this wa...
1,"[ios, xcode, xcode14]",Missing file libarclite_iphoneos.a (Xcode 14.3),<p>After installing Xcode 14.3 in order to run...
2,"[java, android, kotlin, dependencies]",Duplicate class in Kotlin Android,"<p><img src=""https://i.stack.imgur.com/ugp7v.p..."
3,"[assembly, optimization, x86-64, cpu-architect...",Why does this code execute more slowly after s...,"<p>I was reading <a href=""https://en.wikipedia..."
4,"[reactjs, npm, create-react-app, warnings]",Message &quot;npm WARN config global `--global...,"<p>I already installed Node.js on my machine, ..."
5,"[github, heroku]",Heroku and GitHub: &quot;Items could not be re...,<p>I simply want to connect to my GitHub accou...
6,"[linux, git, ubuntu, filesystems, cve-2022-24765]",git submodule update failed with &#39;fatal: d...,<p>I mounted a new hdd in my linux workstation...
7,"[node.js, eslint, webstorm, eslintrc, typescri...",ESLint: TypeError: this.libOptions.parse is no...,<p>I was getting started with Next.js on WebSt...
8,"[reactjs, next.js]",React 18: Hydration failed because the initial...,<p>I'm trying to get SSR working in my app but...
9,"[javascript, reactjs]",Deprecation notice: ReactDOM.render is no long...,<p>I get this error every time I create a new ...


Quota journalier de requêtes restant : 249


**Si on essaie de faire plus de 25 pages, on a l'erreur suivante : *403, access_denied, page above 25 requires access token or app key***

# **Conclusion**

L'API est intéresssante, puisqu'elle permet d'automatiser le processus, mais présente des désavantages : 
- Pour récupérer beaucoup de données, il faut avoir un access token ou une app key.
- Le nombre de requêtes est limité quotidiennement.
- Il n'est pas possible d'appliquer simultanément autant de critères qu'en SQL via le [*StackExchange Data Explorer*](https://data.stackexchange.com/stackoverflow/query/new).